# data preprocessing

In [22]:
import numpy as np
import pandas as pd
import torch
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
# deleting the 'id' column  and Unnamed: 32 column
df.drop(columns=['id', 'Unnamed: 32'], inplace =True)

x = df.iloc[:, 1:]
y = df.iloc[:, 0]

from sklearn.model_selection import train_test_split
x_train, x_test ,y_train, y_test = train_test_split(x,y, test_size=0.2)

from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

import torch
x_train_tensor  = torch.from_numpy(x_train).float()
x_test_tensor   = torch.from_numpy(x_test).float()
y_train_tensor  = torch.from_numpy(y_train).float()
y_test_tensor   = torch.from_numpy(y_test).float()

# Deine Model

In [23]:
import torch.nn as nn


In [33]:
import torch
import torch.nn as nn

class MySimpleNN(nn.Module):
    
    def __init__(self, num_features):

        super().__init__()

        self.network = nn.Sequential(nn.Linear
            (num_features,3),
             nn.ReLU(),
             nn.Linear(3,2),
             nn.ReLU(),
             nn.Linear(2,1),
             nn.Sigmoid()
        )
    
    #Forward pass
    def forward(self, features):

        out = self.network(features)
      

        return out
    


In [34]:
learning_rate =0.1
epochs =25

In [35]:
x_train_tensor.dtype

torch.float32

In [45]:
x_train_tensor.shape

torch.Size([455, 30])

In [36]:
loss_function = nn.BCELoss()

In [37]:
#Create model
model = MySimpleNN(x_train_tensor.shape[1])

#Define optimizer

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)


#define loop
for epoch in range(epochs):

    #Forward pass
    y_pred = model(x_train_tensor)

    #Loss calculation
    loss = loss_function(y_pred, y_train_tensor.reshape(-1,1)) # or .view(-1,1)

     #clear  gradients
    optimizer.zero_grad()

    
    # Backward Pass
    loss.backward()

    #Parameters Update ( weights and bias)
    optimizer.step()


   

    #Print loss in each epochs
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}') 


Epoch 1/25, Loss: 0.6678
Epoch 2/25, Loss: 0.6670
Epoch 3/25, Loss: 0.6662
Epoch 4/25, Loss: 0.6652
Epoch 5/25, Loss: 0.6641
Epoch 6/25, Loss: 0.6627
Epoch 7/25, Loss: 0.6612
Epoch 8/25, Loss: 0.6595
Epoch 9/25, Loss: 0.6577
Epoch 10/25, Loss: 0.6557
Epoch 11/25, Loss: 0.6536
Epoch 12/25, Loss: 0.6512
Epoch 13/25, Loss: 0.6486
Epoch 14/25, Loss: 0.6458
Epoch 15/25, Loss: 0.6427
Epoch 16/25, Loss: 0.6393
Epoch 17/25, Loss: 0.6357
Epoch 18/25, Loss: 0.6317
Epoch 19/25, Loss: 0.6275
Epoch 20/25, Loss: 0.6228
Epoch 21/25, Loss: 0.6177
Epoch 22/25, Loss: 0.6122
Epoch 23/25, Loss: 0.6062
Epoch 24/25, Loss: 0.5997
Epoch 25/25, Loss: 0.5927


In [38]:
#Model evaluation
with torch.no_grad():
    y_pred = model.forward(x_test_tensor)
    y_pred = (y_pred >= 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6320406198501587


In [48]:
!pip install torchinfo


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
from torchinfo import summary

summary(
    model,
    input_size=(1, x_train_tensor.shape[1])  # (batch_size, num_features)
)


Layer (type:depth-idx)                   Output Shape              Param #
MySimpleNN                               [1, 1]                    --
├─Sequential: 1-1                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 3]                    93
│    └─ReLU: 2-2                         [1, 3]                    --
│    └─Linear: 2-3                       [1, 2]                    8
│    └─ReLU: 2-4                         [1, 2]                    --
│    └─Linear: 2-5                       [1, 1]                    3
│    └─Sigmoid: 2-6                      [1, 1]                    --
Total params: 104
Trainable params: 104
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00